# 🧠 Transformando o RAG em Produto: API, WhatsApp e Telegram


## 🔧 Criando uma API com FastAPI para consumir o RAG

In [ ]:
!pip install fastapi uvicorn

Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process
pip install fastapi uvicorn langchain langchain-chroma langchain-huggingface langchain-community
pip install sentence-transformers


In [1]:
!pip install -U langchain-chroma langchain-huggingface


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Eric\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install -U langchain langchain-community langchain-huggingface langchain-chroma

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------------------- -- 2.4/2.5 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 11.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ------ --------------------------------- 2.4/15.5 MB 12.2 MB/s eta 0:00:02
   ------------ --------------------------- 4.7/15.5 MB 11.9 MB/s eta 0:00:01
   ------------------ --------------------- 7.3/15.5 MB 11.9 MB/s eta 0:00:01
   ------------------------- -------------- 10.0/15.5 MB 11.9 MB/s eta 0:00:01
   ------------------------------- -------- 12.3/15.5 MB 11.9 MB/s eta 0:00:01
   -------------------------------------- - 14.9/15.5 MB 11.9 MB/s eta 0:00:01
   ---------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Eric\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_chroma import Chroma  # Atualizado
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint  # Atualizado
from langchain.chains import RetrievalQA
import os

app = FastAPI()

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma(
    persist_directory="./chroma_hf_db",
    embedding_function=embedding
)

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",  # <- ESSA LINHA É ESSENCIAL
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0.3
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

class Consulta(BaseModel):
    pergunta: str

@app.post("/chat/")
def responder(consulta: Consulta):
    resposta = qa.run(consulta.pergunta)
    return {"resposta": resposta}


Executar:
```bash
uvicorn main:app --reload


## 📲 WHATSAPP + IA: Como Integrar com Z-API e Twilio

### ✅ Z-API: Integração com WhatsApp via API Não Oficial

#### O que é Z-API?

Z-API é uma API não oficial do WhatsApp.

Ela funciona usando WhatsApp Web por trás dos panos, simulando o comportamento de um usuário real.

Ideal para projetos rápidos, testes e protótipos, com baixo custo.

#### Como funciona por trás?

Você escanear um QR Code com o WhatsApp Web.

A Z-API cria uma sessão conectada ao seu WhatsApp pessoal ou corporativo.

Sua aplicação então recebe e envia mensagens através dessa conexão.


#### 🛠️ Etapas para usar a Z-API

1. Crie uma conta gratuita:

- Acesse: https://z-api.io

- Faça login com e-mail e senha.


2. Crie uma nova instância:

- Clique em "Nova instância"

- Dê um nome, selecione um plano gratuito ou teste.


3. Escaneie o QR Code:

- Escaneie com seu celular na aba WhatsApp Web.

- Depois disso, sua instância ficará online e conectada ao seu número.


4. Pegue os dados da instância:

- Você verá dois dados importantes:

    - Instance ID
    - Token



#### 📤 Enviando mensagens via Z-API

Para enviar mensagem de volta, você faz um POST HTTP para:

```bash
POST https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text
```

Com o seguinte corpo:

```json 
{
  "phone": "5511999999999",
  "message": "Olá! Recebemos sua dúvida e responderemos em breve."
}



#### 📥 Recebendo mensagens com Webhook

- Z-API envia um POST automático para sua API quando o usuário responde.

- Exemplo de payload que sua API vai receber:

```json
{
  "message": "Olá, quero saber sobre reembolso",
  "phone": "5511999999999",
  "instanceId": "seu_instance",
  "timestamp": 1712271000
}
```



####  Integração com FastAPI:

In [ ]:

from fastapi import FastAPI, Request
import requests

app = FastAPI()

INSTANCE_ID = "sua_instance"
TOKEN = "seu_token"

@app.post("/whatsapp/")
async def receber_mensagem(request: Request):
    dados = await request.json()
    pergunta = dados.get("message")
    numero = dados.get("phone")

    # Aqui entra seu RAG:
    resposta = qa.run(pergunta)

    # Enviar a resposta de volta
    requests.post(
        f"https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text",
        json={
            "phone": numero,
            "message": resposta
        }
    )
    return {"ok": True}


#### 📦 Prós e Contras do Z-API:
![prosecontras](images/prosecontras.png)

### Fluxo da mensagem:

#### 📡 1. Cliente envia mensagem no WhatsApp
Imagine que o cliente digita:

> "Gostaria de saber sobre reembolso." 

Essa mensagem é enviada para o número WhatsApp conectado via Z-API (o seu número pessoal ou empresarial que foi escaneado no painel da Z-API).

#### 🔁 2. Z-API envia um `POST` para o seu webhook FastAPI

No painel da Z-API, você configurou um webhook assim:

``https://seuservidor.com/whatsapp/``

Quando a mensagem chega no WhatsApp, a Z-API chama sua rota /whatsapp/, enviando um JSON com os seguintes dados:

```json
{
  "message": "Gostaria de saber sobre reembolso.",
  "phone": "5511999999999"
}
```

#### 🧠 3. FastAPI recebe e processa no endpoint /whatsapp/

No seu código:

```python
@app.post("/whatsapp/")
async def receber_mensagem(request: Request):
    dados = await request.json()
    pergunta = dados.get("message")
    numero = dados.get("phone")
```

Você está capturando:

- `pergunta` → O conteúdo da mensagem.
- `numero` → O telefone de quem enviou.

#### 🤖 4. Essa pergunta é passada para o seu RAG (LangChain + Chroma + Hugging Face | Ollama )

```python 
resposta = qa.run(pergunta)
```

Aqui, o pipeline acontece:

1. `pergunta` → transformada em vetor.

2. Vetor → busca no Chroma pelos documentos mais próximos.

3. Os documentos recuperados → passados para o LLM (Mixtral via Hugging Face).

4. O modelo gera a resposta final, já com base na sua base de conhecimento.


#### 📤 5. A resposta é enviada de volta ao usuário no WhatsApp

Você faz uma chamada de `POST` para a Z-API:

```python
requests.post(
    f"https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text",
    json={
        "phone": numero,
        "message": resposta
    }
)
```

✅ E o cliente recebe a resposta diretamente no WhatsApp, como se fosse uma pessoa respondendo.

#### RESUMO DO FLUXO VISUAL:
```csharp
[WhatsApp do Cliente]
        ↓
[Z-API Webhook]
        ↓
[FastAPI /whatsapp/] ← captura mensagem
        ↓
[RAG LangChain]
        ↓
[Busca no ChromaDB + LLM]
        ↓
[Resposta gerada]
        ↓
[Z-API envia ao WhatsApp]
        ↓
[Cliente recebe a resposta no WhatsApp]
```


### ✅ Twilio: Integração Oficial com o WhatsApp Business

Twilio é uma plataforma oficialmente aprovada pelo WhatsApp.

É usada por grandes empresas como Nubank, iFood, Mercado Livre.

Você precisa de um número WhatsApp Business verificado e aprovado.

#### 🛠️ Como funciona a integração via Twilio?

1. Criar conta no https://twilio.com/
Obtenha um SID da conta e um Token de autenticação.

2. Obtenha um número de WhatsApp Business sandbox
Vá para: Messaging > Try it out > WhatsApp sandbox

Siga as instruções para conectar seu WhatsApp ao sandbox.

3. Configure seu webhook de recebimento
Twilio enviará mensagens recebidas para um endpoint HTTP (como FastAPI).

#### 📤 Enviando mensagem via Twilio (Python)



In [ ]:
from twilio.rest import Client

account_sid = "SEU_SID"
auth_token = "SEU_TOKEN"
client = Client(account_sid, auth_token)

mensagem = client.messages.create(
    from_='whatsapp:+14155238886',
    body='Olá! Como posso ajudar?',
    to='whatsapp:+5511999999999'
)

print(mensagem.sid)


#### 📥 Recebendo mensagem com webhook (FastAPI)



In [ ]:
from fastapi import FastAPI, Form

app = FastAPI()

@app.post("/twilio/")
async def receber_mensagem(Body: str = Form(...), From: str = Form(...)):
    print(f"Mensagem de {From}: {Body}")
    resposta = qa.run(Body)

    # Aqui você pode usar Twilio para responder de volta
    # (igual ao código acima)
    return "ok"


#### 📦 Prós e Contras do Twilio:
![prosecontras2](images/prosecontras2.png)

In [ ]:


@app.post("/whatsapp/")
def whatsapp_webhook(payload: dict):
    pergunta = payload.get("message")
    resposta = qa.run(pergunta)

    # enviar resposta de volta para o WhatsApp
    requests.post(
        "https://api.z-api.io/instances/SEU_INSTANCE/token/SEU_TOKEN/send-text",
        json={
            "phone": payload["phone"],
            "message": resposta
        }
    )
    return {"status": "ok"}


## 🤖 Conectando ao Telegram Bot API

In [ ]:
!pip install python-telegram-bot


In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, MessageHandler, filters

TOKEN = "SEU_TOKEN_DO_BOT"

async def responder_telegram(update: Update, context: ContextTypes.DEFAULT_TYPE):
    pergunta = update.message.text
    resposta = qa.run(pergunta)
    await update.message.reply_text(resposta)

app = ApplicationBuilder().token(TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, responder_telegram))
app.run_polling()


In [8]:
import requests

url = "https://api.z-api.io/instances/3DF5537109AC2008AEC732C54B267657/token/152FDB1B72705789FF418D22/send-text"

data = {
    "phone": "557193706380",
    "message": "Funcionou via Python com requests!",
    
}

headers = {
    "Content-Type": "application/json",
    "Client-Token":"F069685d93ac046ceaf0109c86552b4f3S"

}

res = requests.post(url, json=data, headers=headers)

print("Status:", res.status_code)
print("Resposta:", res.text)


Status: 200
Resposta: {"zaapId":"3DF64F9B805C40FA931D62CB53C53AFB","messageId":"3EB0412E02ACA2DD181982","id":"3EB0412E02ACA2DD181982"}
